Usefull stuff:

script to clean IMDB reviews and do sentiment analysis
https://github.com/jaypatel00174/moviesentiment_analysis/blob/master/sentiment.py

Hands-On Recommendation Systems with Python
https://www.amazon.com/Hands-Recommendation-Systems-Python-recommendation/dp/1788993756/ref=sr_1_fkmrnull_6?ajr=2&crid=19F7GJ0JYV2AD&keywords=hands-on+recommendation+systems+with+python&linkCode=sl2&linkId=96a355cac564ae8ae2ea24bee599beb7&qid=1547756269&sprefix=hands+on+recommenda%2Caps%2C559&sr=8-6-fkmrnull&tag=tutorialedge-20

ncie script to download movie data from cli
https://github.com/prateekkalra/movie-py-cli/blob/master/movie-py.py

ml on movie ratings
https://www.kaggle.com/juanjotwo/deep-learning-with-python-notebooks-3-5-imdb

the Kaggle site with 1000+ kernels
https://www.kaggle.com/tmdb/tmdb-movie-metadata



### Build a recommender system on IMDB 

https://www.kaggle.com/fabiendaniel/film-recommendation-engine

In [36]:
import pandas as pd
pd.set_option('display.max_columns', 100)
PATH = 'data/'
FILE_CREDITS = 'tmdb_5000_credits.csv'
FILE_MOVIES = 'tmdb_5000_movies.csv'

In [99]:
df = pd.read_csv(PATH + FILE_MOVIES)

In [102]:
# date to Dutch format
df['release_date'] = (df['release_date']
                      .apply(lambda x: pd.to_datetime(x, format = '%Y-%m-%d'))
                      .dt.strftime('%d-%m-%Y'))

In [103]:
import json
json_columns = ['genres', 'keywords', 'production_countries',
                    'production_companies', 'spoken_languages']
# to unquote, remove string
for col in json_columns:
    df[col] = df[col].apply(json.loads)

In [104]:
def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])

In [40]:
# unnest dict_lists and rename columns
df['genre_name'] = df['genres'].apply(lambda x: [d.get('name') for d in x])
df['genre_id'] = df['genres'].apply(lambda x: [d.get('id') for d in x])
df['keywords'] = df['keywords'].apply(lambda x: [d.get('name') for d in x])
df['prod_co_name'] = df['production_companies'].apply(lambda x: [d.get('name') for d in x])
df['prod_co_id'] = df['production_companies'].apply(lambda x: [d.get('id') for d in x])
df['language'] = df['spoken_languages'].apply(lambda x: [d.get('iso_639_1') for d in x])
df['prod_countries'] = df['production_countries'].apply(lambda x: [d.get('iso_3166_1') for d in x])

In [41]:
drop_cols = ['production_companies', 'spoken_languages', 'production_countries', 'genres']
for col in drop_cols:
    df = df.drop(col, axis=1)

In [42]:
df.head(2)

,budget,homepage,id,keywords,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,genre_name,genre_id,prod_co_name,prod_co_id,language,prod_countries
0,237000000,http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,10-12-2009,2787965087,162.0,Released,Enter the World of Pandora.,Avatar,7.2,11800,"[Action, Adventure, Fantasy, Science Fiction]","[28, 12, 14, 878]","[Ingenious Film Partners, Twentieth Century Fo...","[289, 306, 444, 574]","[en, es]","[US, GB]"
1,300000000,http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,19-05-2007,961000000,169.0,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[Adventure, Fantasy, Action]","[12, 14, 28]","[Walt Disney Pictures, Jerry Bruckheimer Films...","[2, 130, 19936]",[en],[US]


In [44]:
def load_tmdb_credits():
    df = pd.read_csv(PATH + FILE_CREDITS)
    json_columns = ['cast', 'crew']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

In [47]:
df_cred = load_tmdb_credits()

In [48]:
df_cred.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."
1,285,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de..."
2,206647,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de..."
3,49026,The Dark Knight Rises,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de..."
4,49529,John Carter,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de..."


In [94]:
for item in df_cred.cast[0]:
    print (item)

{'cast_id': 242, 'character': 'Jake Sully', 'credit_id': '5602a8a7c3a3685532001c9a', 'gender': 2, 'id': 65731, 'name': 'Sam Worthington', 'order': 0}
{'cast_id': 3, 'character': 'Neytiri', 'credit_id': '52fe48009251416c750ac9cb', 'gender': 1, 'id': 8691, 'name': 'Zoe Saldana', 'order': 1}
{'cast_id': 25, 'character': 'Dr. Grace Augustine', 'credit_id': '52fe48009251416c750aca39', 'gender': 1, 'id': 10205, 'name': 'Sigourney Weaver', 'order': 2}
{'cast_id': 4, 'character': 'Col. Quaritch', 'credit_id': '52fe48009251416c750ac9cf', 'gender': 2, 'id': 32747, 'name': 'Stephen Lang', 'order': 3}
{'cast_id': 5, 'character': 'Trudy Chacon', 'credit_id': '52fe48009251416c750ac9d3', 'gender': 1, 'id': 17647, 'name': 'Michelle Rodriguez', 'order': 4}
{'cast_id': 8, 'character': 'Selfridge', 'credit_id': '52fe48009251416c750ac9e1', 'gender': 2, 'id': 1771, 'name': 'Giovanni Ribisi', 'order': 5}
{'cast_id': 7, 'character': 'Norm Spellman', 'credit_id': '52fe48009251416c750ac9dd', 'gender': 2, 'id':

In [52]:
def safe_access(container, index_values):
    # return missing value rather than an error upon indexing/key failure
    result = container
    try:
        for idx in index_values:
            result = result[idx]
        return result
    except IndexError or KeyError:
        return pd.np.nan

In [97]:
df_cred['actor_1_name'] = df_cred['cast'].apply(lambda x: safe_access(x, [1, 'name']))
df_cred['actor_2_name'] = df_cred['cast'].apply(lambda x: safe_access(x, [2, 'name']))
df_cred['actor_3_name'] = df_cred['cast'].apply(lambda x: safe_access(x, [3, 'name']))

In [57]:
def get_director(crew_data):
    directors = [x['name'] for x in crew_data if x['job'] == 'Director']
    return safe_access(directors, [0])

In [98]:
df_cred['director'] = df_cred['crew'].apply(get_director)